In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot


In [5]:
df_news = pd.read_csv("data/Bai_news_headlines.csv")
#df_news.set_index("date", inplace=True)
df_news.date = pd.to_datetime(df_news.date)

In [7]:
df_news.head()

,date,headlines
0,2011-03-29,"PRECIOUS-Gold regains footing on equities, Lib..."
1,2011-03-29,"PRECIOUS-Gold regains footing, talk of cenbank..."
2,2011-03-30,"PRECIOUS-Gold edges up, heads for 10th quarter..."
3,2011-03-30,PRECIOUS-Gold edges down after equity-driven r...
4,2011-03-30,NYMEX-Oil steadies above $104 on US crude stoc...


In [8]:
from textblob import TextBlob


In [9]:
polarity_list=[]
subject_list=[]
for each in df_news['headlines']:
    testimonial = TextBlob(each)
    polarity_list.append(testimonial.sentiment.polarity)
    subject_list.append(testimonial.sentiment.subjectivity)

df_news['polarity']=polarity_list
len(polarity_list)

28220

In [10]:
grouped=df_news.groupby(['date']).mean() # take daily mean

In [11]:
grouped[:10]

,polarity
date,
2011-03-29,0.000000
2011-03-30,-0.063194
2011-03-31,-0.019444
2011-04-01,0.036667
2011-04-03,0.070000
2011-04-04,0.165185
2011-04-05,-0.141342
2011-04-06,0.026667
2011-04-07,0.030476


In [12]:
# TBC: is polarity list still needed?
# for i in range(len(df_news)):
#     date = str(df_news['date'][i])[:10]
#     polarity_list[i] = grouped.loc[date,'polarity']

In [22]:
# len(polarity_list)

28220

In [16]:
import math

In [32]:
grouped.iloc[0].polarity

0.0

In [33]:
cum_senti = []
for current_day_num in range(len(grouped)): # iterate through all days' sentment scores [0,n-1]
    daily_new_senti = grouped.iloc[current_day_num].polarity
    # for current day (t-th day), back track from [0,t-1]
    for back_track_day_num in range(current_day_num):
        # take exponential decay for polarity on i-th day where i:[0,t-1]
        daily_new_senti += grouped.iloc[back_track_day_num].polarity * math.exp(-(current_day_num-back_track_day_num)/7)
    cum_senti.append(daily_new_senti)


* for outer loop, iterate from 1st row (index 0) to last row (index n-1).
* initialize temp var with averaged sentiment score on t-th day, then sum up all back tracked sentiment scores from 1st day (index 0) to 1 day before current day (t-1 th day) where each of them take exponential decay (power must be larger than 0 as current day should not be included in back tracking)
* same shape for cum_senti list and averaged sentiment score list (grouped)


In [34]:
grouped['cum_senti']=cum_senti

In [35]:
new_polarity_list=[]
for i in range(len(df_news)):
    date = str(df_news['date'][i])[:10]
    new_polarity_list.append(grouped.loc[date,'cum_senti'])

In [36]:
df_news['cum_polarity']=new_polarity_list

In [37]:
df_news[:30]

,date,headlines,polarity,cum_polarity
0,2011-03-29,"PRECIOUS-Gold regains footing on equities, Lib...",0.000000,0.000000
1,2011-03-29,"PRECIOUS-Gold regains footing, talk of cenbank...",0.000000,0.000000
2,2011-03-30,"PRECIOUS-Gold edges up, heads for 10th quarter...",0.000000,-0.063194
3,2011-03-30,PRECIOUS-Gold edges down after equity-driven r...,-0.155556,-0.063194
4,2011-03-30,NYMEX-Oil steadies above $104 on US crude stoc...,-0.350000,-0.063194
5,2011-03-30,SOFTS-Cocoa drops as Ivorian rebel gains stoke...,0.000000,-0.063194
6,2011-03-30,"GRAINS-Soybeans rise, corn slips in acreage ba...",0.000000,-0.063194
7,2011-03-30,SOFTS-Cocoa climbs back from 2-1/2 month low,0.000000,-0.063194
8,2011-03-30,"METALS-Copper eases, doubts about China demand...",0.000000,-0.063194
9,2011-03-30,"PRECIOUS-Gold rises along with equities, risk ...",0.000000,-0.063194


In [38]:
df_news.to_csv("news_title_with_2_sentiment_indicators.csv",index=False)